In [1]:
!pip install lifelines
!pip install pyckmeans
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 11.6 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=6aa4986117559e5355d374fcf6faeda9abb613ccfb6f4f12202a236ed5adef21
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyckmeans: filename=pyckmeans-0.9.4-cp310-cp310-linux_x86_64.whl size=93211 sha256=9240821bf2a18005d7536eb635d1fe211ccb0e98c7b5888b1068bebfae1563e7
  Stored in directory: /root/.cache/pip/wheels/c3/83/bb/d0fc31d29f361aff1f400ca2fb2d32c98f900e1aa480229f99
Successfully built pyckmeans
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.2 MB/

In [2]:
import numpy as np
import pandas as pd
#from lifelines import KaplanMeierFitter, CoxPHFitter
from pyckmeans import CKmeans
import matplotlib.pyplot as plt

from lifelines.fitters.coxph_fitter import CoxPHFitter

from google.colab import drive

import wandb
import os

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
# --- Load raw BRCA data ---
drive.mount('/content/gdrive')
!unzip gdrive/MyDrive/TCGA-BRCA_1079.zip

Mounted at /content/gdrive
Archive:  gdrive/MyDrive/TCGA-BRCA_1079.zip
  inflating: TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv  


In [5]:
def drop_elements_from_cluster(cluster, consensus_matrix):
  new_cluster = []
  for j in range(0, len(cluster)):
    if consensus_matrix[cluster[0]][cluster[j]] == 1:
      new_cluster.append(cluster[j])
  return new_cluster

In [6]:
def convert_indices_to_binary_list(number_elements, cluster_0, cluster_1, cluster_2, cluster_3, cluster_4):
  cluster0_column = []
  cluster1_column = []
  cluster2_column = []
  cluster3_column = []
  cluster4_column = []
  for k in range(0, number_elements):
    if k in cluster_0:
      cluster0_column.append(1)
      cluster1_column.append(0)
      cluster2_column.append(0)
      cluster3_column.append(0)
      cluster4_column.append(0)
    elif k in cluster_1:
      cluster0_column.append(0)
      cluster1_column.append(1)
      cluster2_column.append(0)
      cluster3_column.append(0)
      cluster4_column.append(0)
    elif k in cluster_2:
      cluster0_column.append(0)
      cluster1_column.append(0)
      cluster2_column.append(1)
      cluster3_column.append(0)
      cluster4_column.append(0)
    elif k in cluster_3:
      cluster0_column.append(0)
      cluster1_column.append(0)
      cluster2_column.append(0)
      cluster3_column.append(1)
      cluster4_column.append(0)
    elif k in cluster_4:
      cluster0_column.append(0)
      cluster1_column.append(0)
      cluster2_column.append(0)
      cluster3_column.append(0)
      cluster4_column.append(1)
    else:
      cluster0_column.append(0)
      cluster1_column.append(0)
      cluster2_column.append(0)
      cluster3_column.append(0)
      cluster4_column.append(0)

  return [cluster0_column, cluster1_column, cluster2_column, cluster3_column, cluster4_column]

In [7]:
def fit_cox_model(cluster, data_frame):
  print("Original dataframe shape: ", data_frame.shape)
  cluster_frame = pd.DataFrame({'cluster': cluster})
  # Concatenate cluster membership to data table
  if 'cluster' in data_frame.columns.values:
    data_frame = data_frame.drop('cluster', axis = 1) # Drop columns that could have been created previously
  data_frame = pd.concat([data_frame, cluster_frame], axis = 1)

  # Get rows with null entry
  null_mask = data_frame.isnull().any(axis=1)
  null_rows = data_frame[null_mask]
  print(null_rows)

  # Remove rows with null entry
  data_frame.dropna(inplace=True) # remove all rows with any null value
  print("Final dataframe shape: ", data_frame.shape)

  # Fit Cox model
  cph = CoxPHFitter()
  cph.fit(data_frame, duration_col = 'OS.time', event_col = 'OS')
  cph.print_summary()

  # exp(coef) <=> hazard ratio // AN ESTIMATE OF THE TRUE HAZARD RATIO. IT HAS A STANDARD ERROR ASSOCIATED WITH IT.
  # A one unit increase in the covariate will increase the hazard by the hazard ratio

  # Every coefficient comes with a p-value
  # The p-value represents the probability of observing this coefficient in a sample if the null hypothesis was true.
  # The null hypothesis states that the coefficient=0, meaning that the predictor variable does not influence the hazard rate (occurence of the event)

  # The CoxPHFitter computes p-values using the chi-squared test.
  # The reference is in "Survival Analysis by John P. Klein and Melvin L. Moeschberger, Second Edition", page 256

  p_values = cph._compute_p_values()
  hazard_ratios = cph.hazard_ratios_.tolist()
  coefficients_ci = cph.confidence_intervals_
  print(p_values)
  print(hazard_ratios)
  print(coefficients_ci)
  print("p-value:", p_values[4])
  print("hazard-ratio:", hazard_ratios[4])

  return p_values[4], hazard_ratios[4]

In [10]:
for i in range (1, 2):
  data_frame = pd.read_csv("TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv", sep="\t")
  data_frame = data_frame[['age', 'OS', 'OS.time', 'stage_2', 'stage_3', 'stage_4']]

  #name_embedding = "Attention_AE_EM_{i}".format(i=i)
  name_embedding = "UMAP_EM"
  embeddings = np.loadtxt(name_embedding+".txt")
  print("Embeddings shape: ", embeddings.shape)
  print("------------- i =", i, "--------------------")
  number_of_clusters = 5
  ckm = CKmeans(k=number_of_clusters, n_rep=20, p_samp=1, p_feat=1)
  ckm.fit(embeddings)
  ckm_results = ckm.predict(embeddings, linkage_type='average')
  print(ckm_results.cl)
  consensus_matrix = ckm_results.cmatrix
  print(consensus_matrix)

  cluster_0 = []
  cluster_1 = []
  cluster_2 = []
  cluster_3 = []
  cluster_4 = []
  for j in range(0, ckm_results.cl.shape[0]):
    if ckm_results.cl[j] == 1:
      cluster_1.append(j)
    elif ckm_results.cl[j] == 2:
      cluster_2.append(j)
    elif ckm_results.cl[j] == 3:
      cluster_3.append(j)
    elif ckm_results.cl[j] == 4:
      cluster_4.append(j)
    elif ckm_results.cl[j] == 0:
      cluster_0.append(j)
  #print(cluster_0)
  #print(cluster_1)
  #print(cluster_2)
  #print(cluster_3)
  #print(cluster_4)
  print("Cluster sizes: ", len(cluster_0), ", ", len(cluster_1), ", ", len(cluster_2), ", ", len(cluster_3), ", ", len(cluster_4))
  print("Sum of cluster sizes: ", len(cluster_0)+len(cluster_1)+len(cluster_2)+len(cluster_3)+len(cluster_4))

  cluster_0 = drop_elements_from_cluster(cluster_0, consensus_matrix)
  cluster_1 = drop_elements_from_cluster(cluster_1, consensus_matrix)
  cluster_2 = drop_elements_from_cluster(cluster_2, consensus_matrix)
  cluster_3 = drop_elements_from_cluster(cluster_3, consensus_matrix)
  cluster_4 = drop_elements_from_cluster(cluster_4, consensus_matrix)
  print("Cluster sizes after dropping:", len(cluster_0), ", ", len(cluster_1), ", ", len(cluster_2), ", ", len(cluster_3), ", ", len(cluster_4))

  cluster_column_list = convert_indices_to_binary_list(embeddings.shape[0], cluster_0, cluster_1, cluster_2, cluster_3, cluster_4)
  print("Column_lengths: ",  len(cluster_column_list[0]))

  for j in range(0, number_of_clusters):
    if cluster_column_list[j].count(1) > 10:
      print("Cluster size in column form:", cluster_column_list[j].count(1))
      p_value, hazard_ratio = fit_cox_model(cluster_column_list[j], data_frame)
      print("p-value:", p_value)
      print("hazard ratio:", hazard_ratio)
      cluster_name = "k_5_{base}_{j}".format(base=name_embedding, j=j)
      run = wandb.init(project="SURVIVAL_ANALAYSIS_k=5", # Set the project where this run will be logged
                        name=cluster_name
      )

      wandb.log({
                "p_value": p_value,
                "hazard_ratio": hazard_ratio,
                "cluster_size": cluster_column_list[j].count(1)
            })

      if p_value < 0.001:
        file_name = cluster_name + ".txt"
        np.savetxt(file_name, cluster_column_list[j])



Embeddings shape:  (1079, 10)
------------- i = 1 --------------------
[2 1 1 ... 1 1 1]
[[1.   0.   0.   ... 0.   0.   0.1 ]
 [0.   1.   0.95 ... 0.75 0.7  0.65]
 [0.   0.95 1.   ... 0.7  0.75 0.7 ]
 ...
 [0.   0.75 0.7  ... 1.   0.5  0.5 ]
 [0.   0.7  0.75 ... 0.5  1.   0.8 ]
 [0.1  0.65 0.7  ... 0.5  0.8  1.  ]]
Cluster sizes:  183 ,  226 ,  283 ,  198 ,  189
Sum of cluster sizes:  1079
Cluster sizes after dropping: 174 ,  11 ,  92 ,  16 ,  4
Column_lengths:  1079
Cluster size in column form: 174
Original dataframe shape:  (1079, 6)
       age   OS  OS.time  stage_2  stage_3  stage_4  cluster
0     55.0  0.0   4047.0      NaN      NaN      NaN        0
180   81.0  0.0    608.0      NaN      NaN      NaN        0
213   76.0  0.0   1217.0      NaN      NaN      NaN        0
222   76.0  0.0    304.0      NaN      NaN      NaN        0
223   40.0  0.0    304.0      NaN      NaN      NaN        0
225   69.0  0.0     31.0      NaN      NaN      NaN        0
235   68.0  0.0    579.0      N

<lifelines.CoxPHFitter: fitted with 1054 total observations, 915 right-censored observations>
             duration col = 'OS.time'
                event col = 'OS'
      baseline estimation = breslow
   number of observations = 1054
number of events observed = 139
   partial log-likelihood = -748.91
         time fit was run = 2024-03-07 19:41:49 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
age         0.04       1.04       0.01             0.02             0.05                 1.02                 1.05
stage_2     0.60       1.83       0.28             0.05             1.15                 1.05                 3.17
stage_3     1.33       3.80       0.30             0.75             1.92                 2.13                 6.79
stage_4     2.45      11.54       0.37             1.72             3.17                 5.61                23.75
cluster     0.18       1.20       0.24            -0.29             0.66                 0.74                 1.94

            cmp to    z      p   -log2(p)
covariate                                
age           0.00 5.39 <0.005      23.73
stage_2       0.00 2.14   0.03       4.95
stage_3       0.00 4.51 <0.005      17.21
stage_4       0.00 6.64 <0.005      34.90
cluster       0.00 0.75   0.45       1.14
---
Concordance = 0.76
Partial AIC = 1507.82
log-likelihood ratio test = 76.11 on 5 df
-log2(p) of ll-ratio test = 47.38

wandb: Currently logged in as: cosybio-compsysmed. Use `wandb login --relogin` to force relogin


[7.16612802e-08 3.22747228e-02 6.60924727e-06 3.11063393e-11
 4.52813149e-01]
[1.0374576184870778, 1.8254599605503767, 3.7987721643817367, 11.538215007486682, 1.2006049201008315]
           95% lower-bound  95% upper-bound
covariate                                  
age               0.023394         0.050152
stage_2           0.050887         1.152777
stage_3           0.754123         1.915233
stage_4           1.723899         3.167430
cluster          -0.294483         0.660134
p-value: 0.45281314945379547
hazard-ratio: 1.2006049201008315
p-value: 0.45281314945379547
hazard ratio: 1.2006049201008315


Cluster size in column form: 11
Original dataframe shape:  (1079, 6)
       age   OS  OS.time  stage_2  stage_3  stage_4  cluster
0     55.0  0.0   4047.0      NaN      NaN      NaN        0
180   81.0  0.0    608.0      NaN      NaN      NaN        0
213   76.0  0.0   1217.0      NaN      NaN      NaN        0
222   76.0  0.0    304.0      NaN      NaN      NaN        0
223   40.0  0.0    304.0      NaN      NaN      NaN        0
225   69.0  0.0     31.0      NaN      NaN      NaN        0
235   68.0  0.0    579.0      NaN      NaN      NaN        0
397   43.0  1.0   3262.0      NaN      NaN      NaN        0
459   46.0  1.0    749.0      NaN      NaN      NaN        0
463   90.0  1.0   1542.0      NaN      NaN      NaN        0
470   45.0  1.0   2573.0      NaN      NaN      NaN        0
474   61.0  0.0   7777.0      NaN      NaN      NaN        0
479   57.0  1.0   2373.0      NaN      NaN      NaN        0
482   73.0  1.0   3126.0      NaN      NaN      NaN        0
488   58.0  1.0 

...

cluster_size,▁
hazard_ratio,▁
p_value,▁
cluster_size,11
hazard_ratio,0.0
p_value,0.9935


Cluster size in column form: 16
Original dataframe shape:  (1079, 6)
       age   OS  OS.time  stage_2  stage_3  stage_4  cluster
0     55.0  0.0   4047.0      NaN      NaN      NaN        0
180   81.0  0.0    608.0      NaN      NaN      NaN        0
213   76.0  0.0   1217.0      NaN      NaN      NaN        0
222   76.0  0.0    304.0      NaN      NaN      NaN        0
223   40.0  0.0    304.0      NaN      NaN      NaN        0
225   69.0  0.0     31.0      NaN      NaN      NaN        0
235   68.0  0.0    579.0      NaN      NaN      NaN        0
397   43.0  1.0   3262.0      NaN      NaN      NaN        0
459   46.0  1.0    749.0      NaN      NaN      NaN        0
463   90.0  1.0   1542.0      NaN      NaN      NaN        0
470   45.0  1.0   2573.0      NaN      NaN      NaN        0
474   61.0  0.0   7777.0      NaN      NaN      NaN        0
479   57.0  1.0   2373.0      NaN      NaN      NaN        0
482   73.0  1.0   3126.0      NaN      NaN      NaN        0
488   58.0  1.0 

<lifelines.CoxPHFitter: fitted with 1054 total observations, 915 right-censored observations>
             duration col = 'OS.time'
                event col = 'OS'
      baseline estimation = breslow
   number of observations = 1054
number of events observed = 139
   partial log-likelihood = -748.89
         time fit was run = 2024-03-07 19:42:05 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
age         0.04       1.04       0.01             0.02             0.05                 1.02                 1.05
stage_2     0.61       1.83       0.28             0.06             1.16                 1.06                 3.18
stage_3     1.32       3.73       0.30             0.74             1.90                 2.09                 6.66
stage_4     2.42      11.25       0.37             1.70             3.14                 5.48                23.11
cluster     0.41       1.51       0.51            -0.59             1.42                 0.55                 4.12

            cmp to    z      p   -log2(p)
covariate                                
age           0.00 5.37 <0.005      23.59
stage_2       0.00 2.16   0.03       5.01
stage_3       0.00 4.46 <0.005      16.86
stage_4       0.00 6.59 <0.005      34.43
cluster       0.00 0.81   0.42       1.25
---
Concordance = 0.75
Partial AIC = 1507.79
log-likelihood ratio test = 76.14 on 5 df
-log2(p) of ll-ratio test = 47.40

[7.92396483e-08 3.11074961e-02 8.38035162e-06 4.33016077e-11
 4.19944156e-01]
[1.0373374435159837, 1.8327976218022979, 3.730545044943174, 11.250783059107347, 1.5114717962889186]
           95% lower-bound  95% upper-bound
covariate                                  
age               0.023275         0.050039
stage_2           0.055009         1.156678
stage_3           0.737370         1.895739
stage_4           1.700817         3.140059
cluster          -0.590774         1.416942
p-value: 0.4199441555357849
hazard-ratio: 1.5114717962889186
p-value: 0.4199441555357849
hazard ratio: 1.5114717962889186


cluster_size,▁
hazard_ratio,▁
p_value,▁
cluster_size,92
hazard_ratio,0.63348
p_value,0.2141
